# Unit 3 Integrating the CNN Model with Your Drawing App

## Connecting Your Drawing to AI

You've built a web page where users can draw and made the canvas interactive with **JavaScript**. Now, it's time to bring everything together by connecting your drawing canvas to a real **AI model**. In this lesson, you’ll learn how to send your drawing to the server, have a **Convolutional Neural Network (CNN)** analyze it, and display the AI’s guess right on your web page.

### What You'll Learn

In this section, you will:

  * Load a trained **CNN** model in your **Flask** app.
  * Process the drawing sent from the browser so it’s ready for the model.
  * Use the model to predict what the user drew and show the result in the UI.

First, you'll load the trained CNN model in the Flask app like this:

```python
import tensorflow as tf

model = tf.keras.models.load_model('path/to/your/model')
```

This should be done once, typically at the top of your Flask app, so the model is ready to use for predictions.

-----

Next, let's see how the server receives the image from the browser, decodes it, and prepares it for the model:

```python
data_url = request.form.get('image')  # Get the image data sent from the browser (as a data URL)
header, encoded = data_url.split(',', 1)  # Split the data URL to separate the header from the actual image data
img_bytes = base64.b64decode(encoded)  # Decode the base64-encoded image data into bytes
img = Image.open(io.BytesIO(img_bytes))  # Open the image using PIL from the bytes
```

If the image has transparency, you’ll learn how to handle it so the model gets a clean input:

```python
if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
    background = Image.new('RGBA', img.size, (255, 255, 255, 255))  # Create a white background image
    img = Image.alpha_composite(background, img.convert('RGBA'))  # Composite the drawing onto the white background
```

You’ll also see how to convert and resize the image to match the model’s training data:

```python
img = img.convert('L').resize((28, 28))  # Convert to grayscale and resize to 28x28 pixels
```

And how to prepare the image for prediction:

```python
arr = np.array(img, dtype=np.float32) / 255.0  # Convert image to a NumPy array and normalize pixel values to [0, 1]
arr = 1.0 - arr  # Invert colors if needed (so black lines become white, matching training data)
arr = arr.reshape(1, 28, 28, 1)  # Reshape for the model: batch size 1, 28x28 image, 1 channel
```

-----

Finally, you’ll use the model to predict and display the result:

```python
preds = model.predict(arr)  # Get prediction probabilities from the model
idx = np.argmax(preds)  # Find the index of the highest probability (the predicted class)
prediction = categories[idx]  # Get the label for the predicted class
confidence = float(np.max(preds))  # Get the confidence score for the prediction
```

You’ll also learn how to display the AI’s guess and confidence back to the user, making your app feel smart and interactive.

### Why This Is Exciting

This is where your project truly comes to life. By integrating your UI with a real AI model, you’re not just building a drawing app—you’re creating a tool that can “see” and “understand” what users draw. This skill is valuable for many modern applications, from games to educational tools to real-world AI products.

Let’s get started and see your AI in action\!




## Processing Images for AI Recognition

When users draw on the canvas and submit their drawing, the image is sent to the server as a PNG. PNG images can include an alpha channel (transparency), which can cause issues when processing the image for AI recognition. To ensure consistent results, you need to check if the uploaded image has an alpha channel and, if so, composite it over a white background to remove transparency.

Your task: In app/main.py, complete the code that checks if the uploaded image has an alpha channel (such as mode "RGBA" or "LA"). If it does, create a new white background image and paste the original image onto it using the alpha channel as a mask. This will ensure the image is fully opaque and ready for further processing by the AI model.


```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # TODO: Check if the image has an alpha channel (RGBA or LA) or if it's a palette image with transparency (img.mode is 'P' and 'transparency' in img.info exists)
        if img.mode in ______ or (img.mode == ______ and ______ in img.info):
            # TODO: Create a new white background image using Image.new with mode 'RGBA', original image size, and white color (255, 255, 255, 255)
            background = ______
            img = Image.alpha_composite(background, img.convert('RGBA'))

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)

```

```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Check if the image has an alpha channel (RGBA or LA) or if it's a palette image with transparency (img.mode is 'P' and 'transparency' in img.info exists)
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            # Create a new white background image using Image.new with mode 'RGBA', original image size, and white color (255, 255, 255, 255)
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)

```

## Preparing Images for AI Recognition

Great job handling the transparency in your images! Now, we need to prepare them for our AI model. Neural networks that process images typically expect a specific format — in our case, grayscale images of a standard size.

Your task: Add code to convert the image to grayscale using the 'L' mode and resize it to 28x28 pixels. This is the exact input size our model expects to make accurate predictions.

You'll also need to create a second processed image from the original bytes for comparison purposes. This step helps us verify that our preprocessing is working correctly.


```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # TODO: Convert the image to grayscale using the 'L' mode and resize it to 28x28 pixels
        img = img.convert(______).resize(______)

    return render_template('index.html',
                           prediction=prediction,
                           confidence=None)

if __name__ == '__main__':
    app.run(debug=True)

```

I've updated the Python code to handle the image preprocessing as you requested. This includes converting the image to grayscale and resizing it to 28x28 pixels, which is the exact input size your AI model expects.

I've also added logic to prepare two versions of the image for comparison: the image after transparency handling (your "original" for visual reference) and the final 28x28 grayscale image after all preprocessing steps. These will be passed to your HTML template for display, allowing you to visually verify that the preprocessing is working correctly.

```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None
    # Variables to hold base64 image data for display in the frontend
    original_image_display_data = None
    processed_image_display_data = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        # This ensures the background is white before further processing
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Store the image state after transparency handling for display as "original"
        # This allows comparison with the final processed image
        buffered_original = io.BytesIO()
        img.save(buffered_original, format="PNG")
        original_image_display_data = base64.b64encode(buffered_original.getvalue()).decode('utf-8')

        # Create a copy of the image to perform further processing for the model
        img_for_model = img.copy()

        # Convert the image for the model to grayscale using the 'L' mode
        # 'L' mode means 8-bit pixels, black and white
        # Then, resize it to 28x28 pixels, which is the required input size for the model
        img_for_model = img_for_model.convert('L').resize((28, 28))

        # Store the processed image state for display as "processed"
        # This shows the exact image that will be fed into the neural network
        buffered_processed = io.BytesIO()
        img_for_model.save(buffered_processed, format="PNG") # Save as PNG for display purposes
        processed_image_display_data = base64.b64encode(buffered_processed.getvalue()).decode('utf-8')

        # Prepare the image array for the TensorFlow model:
        # 1. Convert PIL Image to NumPy array
        # 2. Normalize pixel values from [0, 255] to [0, 1]
        # 3. Add a channel dimension (for grayscale, it's 1)
        # 4. Add a batch dimension (for a single image prediction, it's 1)
        img_array = np.array(img_for_model) / 255.0
        img_array = np.expand_dims(img_array, axis=-1) # Adds the channel dimension (28, 28) -> (28, 28, 1)
        img_array = np.expand_dims(img_array, axis=0)  # Adds the batch dimension (28, 28, 1) -> (1, 28, 28, 1)

        # Make prediction using the loaded TensorFlow model
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        prediction = categories[predicted_class_index]
        confidence = float(np.max(predictions)) * 100 # Convert confidence to a percentage

    # Render the HTML template, passing prediction results and image data for display
    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence,
                           original_image_data=original_image_display_data,
                           processed_image_data=processed_image_display_data)

if __name__ == '__main__':
    # Run the Flask application in debug mode
    app.run(debug=True)


```

Since the app.py now passes original_image_data and processed_image_data to the template, you'll also need an index.html file to display these images along with the prediction. Here's a basic index.html that you can use:
Basic HTML Template (index.html)
Aug 2, 12:48 PM


With these changes, your Flask application will now correctly preprocess the images for your AI model and provide visual feedback on the preprocessing steps.

Let me know if you'd like to refine the display of the images or explore other preprocessing techniques!

## Creating Debug Images for AI Visualization

Now that we've prepared our images for the AI model, let's add a debugging feature. When developing image recognition systems, it's crucial to see exactly what the model "sees" after all our preprocessing steps.

In this task, you'll create a debug visualization by saving the processed image as a PNG file. This allows us to display the processed image in our web interface, helping us understand how our preprocessing affects the drawings.

Your task: Complete the code to convert the processed image to RGB (so it can be saved as a color PNG) and save it at static/debug_image.png. This file will be shown in the UI for debugging.


```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Convert, resize to match training
        img = img.convert('L').resize((28, 28))

        # TODO: Save debug image to static folder with name 'debug_image.png'
        debug_path = os.path.join(______, ______)
        img.save(______)

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)

```

I've updated the app.py file to include the debugging feature you requested. Now, after preprocessing, the 28x28 grayscale image will be converted to RGB and saved as static/debug_image.png. This allows you to visually inspect the exact input your AI model receives.

I've also adjusted the index.html file to display this debug_image.png from the static folder, replacing the previous base64 image displays for a cleaner approach.

Here's the updated app.py code:

Here's the updated index.html to display the debug image:

Now you can draw on the canvas, click "Predict," and see the 28x28 grayscale image that the model actually uses for prediction, located in the static/debug_image.png file. This will be very helpful for debugging your image recognition system!


```python
from flask import Flask, render_template, request, url_for
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io
import os

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None
    debug_image_url = None # To pass the URL of the debug image to the template

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Convert, resize to match training
        img = img.convert('L').resize((28, 28))

        # Ensure the static directory exists
        if not os.path.exists(app.static_folder):
            os.makedirs(app.static_folder)

        # Convert the processed image to RGB before saving as PNG for better visualization
        # Even though it's grayscale, saving as RGB allows broader compatibility for viewers.
        img_rgb = img.convert('RGB')

        # Save debug image to static folder with name 'debug_image.png'
        debug_path = os.path.join(app.static_folder, 'debug_image.png')
        img_rgb.save(debug_path)
        debug_image_url = url_for('static', filename='debug_image.png') # Generate URL for the image

        # Prepare the image array for the TensorFlow model:
        # 1. Convert PIL Image to NumPy array
        # 2. Normalize pixel values from [0, 255] to [0, 1]
        # 3. Add a channel dimension (for grayscale, it's 1)
        # 4. Add a batch dimension (for a single image prediction, it's 1)
        img_array = np.array(img) / 255.0 # Use the grayscale img here for the model
        img_array = np.expand_dims(img_array, axis=-1) # Adds the channel dimension (28, 28) -> (28, 28, 1)
        img_array = np.expand_dims(img_array, axis=0)  # Adds the batch dimension (28, 28, 1) -> (1, 28, 28, 1)

        # Make prediction using the loaded TensorFlow model
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        prediction = categories[predicted_class_index]
        confidence = float(np.max(predictions)) * 100 # Convert confidence to a percentage

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence,
                           debug_image_url=debug_image_url) # Pass the URL

if __name__ == '__main__':
    app.run(debug=True)


```

## Preparing Images for Neural Network Analysis

Great progress! With our image saved for debugging, we now need to transform it into a format our neural network can understand.

Neural networks don't process images directly — they work with numerical arrays. For our drawing recognition model, we need to convert the image into a normalized array, invert the colors (since many models are trained on white drawings on black backgrounds), and reshape it to match the model's expected input dimensions.

Your task:

Complete the code that prepares the image for prediction by converting it to a normalized NumPy array, inverting the colors, and reshaping it to the correct dimensions for our model.

```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io, os

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Convert, resize to match training
        img = img.convert('L').resize((28, 28))

        # Save debug image to static folder
        debug_path = os.path.join(app.static_folder, 'debug_image.png')
        img.save(debug_path)

        # TODO: Prepare image for prediction
        # 1. Convert image to numpy array with float32 data type and normalize by dividing by 255.0
        arr = np.array(______, dtype=______) / ______
        # 2. Invert the colors (1.0 - arr)
        arr = ______ - arr
        # 3. Reshape to match model input dimensions (batch_size, height, width, channels)
        arr = arr.reshape(______, ______, ______, ______)

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)


```

I've updated the app.py Canvas to include the code for preparing the image for neural network analysis. This involves converting the image to a normalized NumPy array, inverting its colors, and reshaping it to the correct dimensions for your model.

Here's the updated app.py code:


```python
from flask import Flask, render_template, request, url_for
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io
import os

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None
    debug_image_url = None # To pass the URL of the debug image to the template

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Convert, resize to match training
        img = img.convert('L').resize((28, 28))

        # Ensure the static directory exists
        if not os.path.exists(app.static_folder):
            os.makedirs(app.static_folder)

        # Convert the processed image to RGB before saving as PNG for better visualization
        # Even though it's grayscale, saving as RGB allows broader compatibility for viewers.
        img_rgb = img.convert('RGB')

        # Save debug image to static folder with name 'debug_image.png'
        debug_path = os.path.join(app.static_folder, 'debug_image.png')
        img_rgb.save(debug_path)
        debug_image_url = url_for('static', filename='debug_image.png') # Generate URL for the image

        # Prepare image for prediction
        # 1. Convert image to numpy array with float32 data type and normalize by dividing by 255.0
        arr = np.array(img, dtype=np.float32) / 255.0
        # 2. Invert the colors (1.0 - arr)
        arr = 1.0 - arr
        # 3. Reshape to match model input dimensions (batch_size, height, width, channels)
        arr = arr.reshape(1, 28, 28, 1)

        # Make prediction using the loaded TensorFlow model
        predictions = model.predict(arr) # Use the prepared 'arr' for prediction
        predicted_class_index = np.argmax(predictions)
        prediction = categories[predicted_class_index]
        confidence = float(np.max(predictions)) * 100 # Convert confidence to a percentage

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence,
                           debug_image_url=debug_image_url) # Pass the URL

if __name__ == '__main__':
    app.run(debug=True)


```

You've successfully prepared your drawing data for the AI model! Now comes the most exciting part — making the model actually analyze your drawing and tell you what it thinks you've drawn.

In this task, you'll implement the code that uses the neural network to make predictions. The model will analyze your processed drawing and determine which of the 12 categories it most closely resembles.

Your task is to complete the code that sends the prepared image array to the model, finds which category has the highest probability score, retrieves the category name, and calculates the confidence level of the prediction.

This is the final step that brings your drawing recognition app to life — connecting the user's input to the AI's analysis and displaying the results

```python
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io, os

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Convert, resize to match training
        img = img.convert('L').resize((28, 28))

        # Save debug image to static folder
        debug_path = os.path.join(app.static_folder, 'debug_image.png')
        img.save(debug_path)

        # Prepare image for prediction - Convert to numpy array and normalize
        arr = np.array(img, dtype=np.float32) / 255.0
        arr = 1.0 - arr
        arr = arr.reshape(1, 28, 28, 1)

        # TODO: Use the model to predict what was drawn
        preds = model.predict(________)
        
        # TODO: Find the index of the category with highest probability
        idx = np.argmax(________)
        
        # TODO: Get the actual category name from our list
        prediction = categories[________]
        
        # TODO: Get the confidence value (how sure the model is)
        confidence = float(np.max(________))

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence)

if __name__ == '__main__':
    app.run(debug=True)

```

I've updated the app.py Canvas to complete the prediction logic. The code now uses your neural network model to analyze the processed drawing, identify the category with the highest probability, and calculate the confidence level of the prediction.

Here's the updated app.py code:

```python
from flask import Flask, render_template, request, url_for
import tensorflow as tf
import numpy as np
from PIL import Image
import base64, io
import os

app = Flask(__name__)
model = tf.keras.models.load_model(
    '/usercode/FILESYSTEM/app/models/drawing_cnn.keras'
)

categories = [
    'house', 'apple', 'car', 'cat',
    'dog', 'flower', 'star', 'tree',
    'bowtie', 'eyeglasses', 'door', 'umbrella'
]

@app.route('/', methods=['GET', 'POST'])
def home():
    prediction = None
    confidence = None
    debug_image_url = None # To pass the URL of the debug image to the template

    if request.method == 'POST':
        # Get base64 image data from canvas
        data_url = request.form.get('image')

        # Decode raw image bytes
        header, encoded = data_url.split(',', 1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes))

        # Composite transparent to white if needed
        if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
            background = Image.new('RGBA', img.size, (255, 255, 255, 255))
            img = Image.alpha_composite(background, img.convert('RGBA'))

        # Convert, resize to match training
        img = img.convert('L').resize((28, 28))

        # Ensure the static directory exists
        if not os.path.exists(app.static_folder):
            os.makedirs(app.static_folder)

        # Convert the processed image to RGB before saving as PNG for better visualization
        # Even though it's grayscale, saving as RGB allows broader compatibility for viewers.
        img_rgb = img.convert('RGB')

        # Save debug image to static folder with name 'debug_image.png'
        debug_path = os.path.join(app.static_folder, 'debug_image.png')
        img_rgb.save(debug_path)
        debug_image_url = url_for('static', filename='debug_image.png') # Generate URL for the image

        # Prepare image for prediction - Convert to numpy array and normalize
        arr = np.array(img, dtype=np.float32) / 255.0
        arr = 1.0 - arr
        arr = arr.reshape(1, 28, 28, 1)

        # Use the model to predict what was drawn
        preds = model.predict(arr)
        
        # Find the index of the category with highest probability
        idx = np.argmax(preds)
        
        # Get the actual category name from our list
        prediction = categories[idx]
        
        # Get the confidence value (how sure the model is)
        confidence = float(np.max(preds)) * 100 # Multiply by 100 to get percentage

    return render_template('index.html',
                           prediction=prediction,
                           confidence=confidence,
                           debug_image_url=debug_image_url) # Pass the URL

if __name__ == '__main__':
    app.run(debug=True)


```

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Drawing App</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            padding-top: 20px;
        }

        canvas {
            border: 2px solid #333;
            cursor: crosshair;
        }

        button {
            padding: 10px 20px;
            margin-top: 10px;
            cursor: pointer;
        }
    </style>
</head>
<body>
    <h2>✏️ Draw</h2>
    <form method="POST" onsubmit="return submitCanvas();">
        <canvas id="canvas" width="280" height="280"></canvas><br>
        <input type="hidden" name="image" id="imageData">
        <button type="button" onclick="clearCanvas()">Clear</button>
        <button type="submit">Guess!</button>
    </form>

    {% if prediction is not none %}
    <div class="result">
        <h3>🤖 AI Guess: <span>{{ prediction }}</span></h3>
        <p>Confidence: {{ confidence|round(2) }}%</p>
        <img src="static/debug_image.png" alt="Your drawing">
    </div>
    {% endif %}
    <script src="{{ url_for('static', filename='script.js') }}"></script>
</body>
</html>

```

```js
const canvas = document.getElementById('canvas');
const ctx = canvas.getContext('2d');
ctx.lineWidth = 5;
ctx.lineCap = 'round';

let drawing = false;

canvas.addEventListener('mousedown', (e) => {
    drawing = true;
    console.log('Started drawing');
    ctx.beginPath();
    ctx.moveTo(e.offsetX, e.offsetY);
});

canvas.addEventListener('mousemove', (e) => {
    if (drawing) {
        ctx.lineTo(e.offsetX, e.offsetY);
        ctx.stroke();
    }
});

canvas.addEventListener('mouseup', () => {
    drawing = false;
    console.log('Stopped drawing');
});

canvas.addEventListener('mouseleave', () => drawing = false);

function clearCanvas() {
    ctx.clearRect(0, 0, canvas.width, canvas.height);
}

function submitCanvas() {
    const dataURL = canvas.toDataURL();
    document.getElementById('imageData').value = dataURL;
    return true;
}
```